<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Hermite Interpolator Tutorial in NRPy 

## Author: Maria C. Babiuc-Hamilton
### Template courtesy Zach Etienne and Brandon Clark

### NRPy+ Source Code for this module: [hermite_interpolator.py](../edit/hermite_interpolator.py)


<a id='toc'></a>

# Table of Contents \[Back to [top](#toc)\]
$$\label{toc}$$ 

This notebook is organized as follows

1. [Hermite1D](#hi1d): Cubic Hermite Interpolation on a 1D Uniform Grid
1. [Hermite2D](#hi2d): Bicubic Hermite Interpolation on a 2D Uniform Grid
1. [Step 1](#himodule): The Hermite Interpolator NRPy+ module
    1. [Step 1.a](#hicoeffs_func): The `compute_hicoeffs_histencl()` function
        1. [Step 1.a.i](#exercise): Exercise: Using `compute_hicoeffs_histencl()`
    1. [Step 1.b](#hioutputc): The  `HI_outputC()` function
1. [Step 2](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF    

<a id='hi1d'></a>

# Hermite Interpolation on a 1D Uniform Grid \[Back to [top](#toc)\]
$$\label{hi1d}$$

In this tutorial, we will develop the interpolation technique using [Hermite interpolation](https://en.wikipedia.org/wiki/Bicubic_interpolation) (HI) techniques. 

We will develop the cubic Hermite interpolation on a *uniform* numerical grid.
Let's start in one dimension, for proof of principle. 
Since the grid is uniform, the spacing between successive grid points is constant and will denote it $h_{x}$. 
Then, given a function $f(x)$ on this uniform grid, we will adopt the notation

$$
f(x_k) = f_k, ~
\frac{df(x_k)}{dx1} = d_x f_k.
$$

We want to interpolate this function onto a point $(x)$ somewhere on the line formed by the edges: $[x_{k}, x_{k+1}]$.

We will need $4$ known points, on a linear grid, starting from the left to the right: 
$$[x_{k-1}, x_{k}, x_{k+1}, x_{k+2}]$$
We define the distance from each known point to the interpolating point: 
$$s(x) = x - x_{k} = \xi h_x$$
where we impose $h_x=const$ and $0<\xi<1.$

HI techniques are usually constructed as follows:
We fit a smooth polynomial *to both the function and its derivative* at the point $x$:
$$
P(x) = f(x) = \sum_{i=0}^3 c_{i} s^i \\
\frac{d}{dx}P(x) = \frac{df(x)}{dx} = \sum_{i=1}^3 i c_{i} s^{i-1}.
$$

The interpolation problem consists of solving this system, to determe the $4$ unknown coefficients:
$$c_{0}, c_{1}, c_{2}, c_{3}.$$
To accomplish this, we form the system of $4$ equations by writing the above equations at the edges:
$$
P(x_{k})             = f_k   = \sum_{i=0}^3 c_{i} s(x_{k})^i \\
P(x_{k+1})           = f_{k+1} = \sum_{i=0}^3 c_{i} s(x_{k+1})^i \\
\frac{d}{dx}P(x_{k}) = d_x f_k = \sum_{i=1}^3 i c_{i} s(x_{k})^{i-1} \\
\frac{d}{dx}P(x_{k+1})= d_x f_{k+1}= \sum_{i=1}^3 i c_{i} s(x_{k+1})^{i-1}.
$$
Noting that: $s(x_{k}) = x_{k} - x_{k} = 0$ and $s(x_{k+1}) = x_{k+1} - x_{k} = h_x$, thus the system of 4 equations with 4 unknowns simplifies to:
$$ 
\left\{
    \begin{array}\\
f_k         &= c_{0}\\
f_{k+1}     &= c_{0} + c_{1}h + c_{2}h^2 + c_{3}h^3\\
d_x f_k     &= c_{1}\\
d_x f_{k+1} &= c_{1} + 2c_{2}h + 3c_{3}h^2
\end{array}
\right.
$$

There are multiple ways to compute the Hermite interpolation coefficients $c_i$.
We solve this system in Mathematica and obtain the solutions:
$$
\begin{array} \\
c_0 &= f_k \\
c_1 &= d_x f_k \\
c_2 &= \frac{3 f_{k+1} - 3 f_{k} - 2 d_x f_k h_x - d_x f_{k+1} h_x}{h_x^2} \\
c_3 &= \frac{- 2 f_{k+1} + 2 f_{k} + d_x f_k h_x + d_x f_{k+1} h_x}{h_x^3}.
\end{array}
$$

The key now is to impose the condition onto the numerical derivative of the function at the edges, such that the function stays smooth, at least locally.
This condition can be written as: 
$$ 
d_x f_{k} = \left\{
\begin{array}\\
0     & \mbox{if } \ \text{sgn}(\delta x_{k−1})\neq \text{sgn}(\delta x_{k})\\
dx_{k}& \mbox{if } \ \text{sgn}(\delta x_{k−1})=\text{sgn}(\delta x_{k})
\end{array}
\right.
$$
and
$$ 
d_x f_{k+1} = \left\{
\begin{array}\\
0        & \mbox{if } \ \text{sgn}(\delta x_{k})\neq \text{sgn}(\delta x_{k+1})\\
dx_{k+1} & \mbox{if } \ \text{sgn}(\delta x_{k})=\text{sgn}(\delta x_{k+1}).
\end{array}
\right.
$$

The conditions above make use of the following quantities that must be calculated: 
$$
\begin{array} \\
\delta x_{k-1} = \frac{f_{k  }-f_{k-1}}{h_x},\\
\delta x_{k  } = \frac{f_{k+1}-f_{k  }}{h_x},\\
\delta x_{k+1} = \frac{f_{k+2}-f_{k+1}}{h_x}
\end{array}
$$
and:
$$
\begin{array} \\
dx_{k  } =\frac{2 \delta x_{k  } \delta x(k-1)}{\delta x_{k  } + \delta x_{k-1}},\\
dx_{k+1} =\frac{2 \delta x_{k+1} \delta x{k  }}{\delta x_{k+1} + \delta x_{k  }}.
\end{array}
$$

Once the slopes are known, the derivatives of the functions are replaced with:
$$d_x f_{k} = dx_k,~d_x f_{k+1} = d x_{k+1}.$$

We also substitute 
$$ 
f_{k+1} = f_k + h_x \delta x_k,~
s(x) =\xi h_x
$$
to simplify the expressions for the Hermite coefficients. 
Now we form the interpolation polynomial:
$$
f(x) = P(x) = \sum_{i=0}^3 c_i s^i = \sum_{i=0}^3 C_i \xi^i
$$ 
where the final form of the 1D Hermite coefficients is:
$$
\begin{array} \\
C_0 &= f_k \\
C_1 &= h_x d x_k\\
C_2 &=  -2 h_x d x_k - h_x d x_{k+1} + 3 h_x \delta x_k\\
C_3 &= h_x d x_k + h_x d x_{k+1} - 2 h_x \delta x_k.
\end{array}
$$
**We must simply compute $(f_k, \delta x_k, d x_k)$ at each point on the grid, impose the continuity condition, and then compute the coefficients $C_i$.** 
<!--- -->

**Recommended: Learn more about the algorithm NRPy+ adopts to automatically compute finite difference derivatives: ([How NRPy+ Computes Finite Difference Coefficients](Tutorial-How_NRPy_Computes_Finite_Difference_Coeffs.ipynb))**


<a id='hi2d'></a>

# Hermite Interpolation on a 2D Uniform Grid \[Back to [top](#toc)\]
$$\label{hi2d}$$

A *bicubic* Hermite interpolation extends the 1D cubic interpolation to a 2D grid. 

We consider a stencil of $4\times 4=16$ known points, on a uniform 2D grid, starting from the lower left corner us, and then advancing right: 


$$
[x_{k-1},y_{k-1}, x_{k-1},y_{k}, x_{k-1},y_{k+1}, x_{k-1},y_{k+2}, \\
 x_{k  },y_{k-1}, x_{k  },y_{k}, x_{k  },y_{k+1}, x_{k  },y_{k+2}, \\
 x_{k+1},y_{k-1}, x_{k+1},y_{k}, x_{k+1},y_{k+1}, x_{k+1},y_{k+2}, \\
 x_{k+1},y_{k-1}, x_{k+2},y_{k}, x_{k+2},y_{k+1}, x_{k+2},y_{k+1}] .
$$

We interpolate onto the 2D point $(x,y)$ in the middle of the rectangle formed by the $4$ corners starting lower-left and going counterclockwise: 
$$
[x_{k  },y_{k  }, x_{k  },y_{k+1},\\
 x_{k+1},y_{k+1}, x_{k+1},y_{k  }].
 $$

The Hermite interpolator is:
$$P(x,y) = \sum_{i=0}^3 \sum_{j=0}^3 c_{ij} s_x^i s_y^j,$$
where $s_x$ is the distance in the $x$ direction from the point of interpolation $(x,y)$ to $x_k$, namely: 
$$s_x(x) = x - x_k = \xi h_x  $$
and $s_y$ is the distance in the $y$ direction from point of interpolation $(x,y)$ to $y_k$: 
$$s_y(y) = y - y_k = \mu h_y.$$

We now form the system of $16$ equations with 16 unknowns. We have $4$ equations for the corners:
$$
P(x_{k  },y_{k  }) = f_{k  ,k  } = \sum_{i=0}^3 \sum_{j=0}^3 c_{ij} s_x(x_{k  })^i s_y(y_{k  })^j,\\
P(x_{k  },y_{k+1}) = f_{k  ,k+1} = \sum_{i=0}^3 \sum_{j=0}^3 c_{ij} s_x(x_{k  })^i s_y(y_{k+1})^j\\
P(x_{k+1},y_{k+1}) = f_{k+1,k+1} = \sum_{i=0}^3 \sum_{j=0}^3 c_{ij} s_x(x_{k+1})^i s_y(y_{k+1})^j\\
P(x_{k+1},y_{k  }) = f_{k+1,k  } = \sum_{i=0}^3 \sum_{j=0}^3 c_{ij} s_x(x_{k+1})^i s_y(y_{k  })^j.
$$

Next we have $4$ equations for the $x$ derivative at the $4$ corners:
$$
\frac{d}{dx}P(x_{k  },y_{k  }) = d_x f_{k  ,k  } = \sum_{i=1}^3 \sum_{j=0}^3 i c_{ij} 
s_x(x_{k  })^{i-1} s_y(y_{k  })^j\\
\frac{d}{dx}P(x_{k  },y_{k+1}) = d_x f_{k  ,k+1} = \sum_{i=1}^3 \sum_{j=0}^3 i c_{ij} 
s_x(x_{k  })^{i-1} s_y(y_{k+1})^j\\
\frac{d}{dx}P(x_{k+1},y_{k+1}) = d_x f_{k+1,k+1} = \sum_{i=1}^3 \sum_{j=0}^3 i c_{ij} 
s_x(x_{k+1})^{i-1} s_y(y_{k+1})^j\\
\frac{d}{dx}P(x_{k+1},y_{k  }) = d_x f_{k+1,k  } = \sum_{i=1}^3 \sum_{j=0}^3 i c_{ij} 
s_x(x_{k+1})^{i-1} s_y(y_{k  })^j
$$

We are adding now $4$ equations for the $y$ derivative at the $4$ corners:
$$\frac{d}{dy}P(x_{k  },y_{k  }) = f_y(k,k) = \sum_{i=0}^3 \sum_{j=1}^3 j c_{ij} s_x(k)^{i} s_y(k)^{j-1}$$
$$\frac{d}{dy}P(x_{k  },y_{k+1}) = f_y(k,k+1) = \sum_{i=0}^3 \sum_{j=1}^3 j c_{ij} s_x(k)^{i} s_y(k+1)^{j-1}$$
$$\frac{d}{dy}P(x_{k+1},y_{k+1}) = f_y(k+1,k+1) = \sum_{i=0}^3 \sum_{j=1}^3 j c_{ij} s_x(k+1)^{i} s_y(k+1)^{j-1}$$
$$\frac{d}{dy}P(x_{k+1},y_{k  }) = f_y(k+1,k) = \sum_{i=0}^3 \sum_{j=1}^3 j c_{ij} s_x(k+1)^{i} s_y(k)^{j-1}$$


Up to now we have $12$ equations. The last $4$ equations are obtained by adding to this the mixed derivatives: 
$\frac{d^2}{dxdy}P(x,y) = f_{xy} = \sum_{i=1}^3 \sum_{j=1}^3 ij c_{ij} s_x^{i-1} s_y^{j-1}$, which becomes at the $4$ corners:

$$\frac{d^2}{dxdy}P(x_{k  },y_{k  }) = f_{xy}(k,k) = \sum_{i=1}^3 \sum_{j=1}^3 ij c_{ij} s_x(k)^{i-1} s_y(k)^{j-1}$$
$$\frac{d^2}{dxdy}P(x_{k  },y_{k+1}) = f_{xy}(k,k+1) = \sum_{i=1}^3 \sum_{j=1}^3 ij c_{ij} s_x(k)^{i-1} s_y(k+1)^{j-1}$$
$$\frac{d^2}{dxdy}P(x_{k+1},y_{k+1}) = f_{xy}(k+1,k+1) = \sum_{i=1}^3 \sum_{j=1}^3 ij c_{ij} s_x(k+1)^{i-1} s_y(k+1)^{j-1}$$
$$\frac{d^2}{dxdy}P(x_{k+1},y_{k  }) = f_{xy}(k+1,k) = \sum_{i=1}^3 \sum_{j=1}^3 ij c_{ij} s_x(k+1)^{i-1} s_y(k)^{j-1}$$


 # TODO: the text below is the original from Tutorial-Finite_Difference_derivatives.ipynb

<a id='fdmodule'></a>

# Step 1: The finite_difference NRPy+ module \[Back to [top](#toc)\]
$$\label{fdmodule}$$

The finite_difference NRPy+ module contains one parameter:

* **FD_CENTDERIVS_ORDER**: An integer indicating the requested finite difference accuracy order (not the order of the derivative) , where FD_CENTDERIVS_ORDER = [the size of the finite difference stencil in each direction, plus one].

The finite_difference NRPy+ module contains two core functions: `compute_fdcoeffs_fdstencl()` and `FD_outputC()`. The first is a low-level function normally called only by `FD_outputC()`, which computes and outputs finite difference coefficients and the numerical grid indices (stencil) corresponding to each coefficient:

<a id='fdcoeffs_func'></a>

## Step 1.a:  The `compute_fdcoeffs_fdstencl()` function \[Back to [top](#toc)\]
$$\label{fdcoeffs_func}$$

**compute_fdcoeffs_fdstencl(derivstring,FDORDER=-1)**:
* Output nonzero finite difference coefficients and corresponding numerical stencil as lists, using as inputs:
    * **derivstring**: indicates the precise type and direction derivative desired:
        * **Centered derivatives**, where the center of the finite difference stencil corresponds to the point where the derivative is desired:
            * For a first-order derivative, set derivstring to "D"+"dirn", where "dirn" is an integer denoting direction. For a second-order derivative, set derivstring to "DD"+"dirn1"+"dirn2", where "dirn1" and "dirn2" are integers denoting the direction of each derivative. Currently only $1 \le N \le 2$ supported (extension to higher-order derivatives is straightforward). Examples in 3D Cartesian coordinates (x,y,z):
                * the derivative operator $\partial_x^2$ corresponds to derivstring = "DD00"
                * the derivative operator $\partial_x \partial_y$ corresponds to derivstring = "DD01"
                * the derivative operator $\partial_z$ corresponds to derivstring = "D2"
        * **Up- or downwinded derivatives**, where the center of the finite difference stencil is *one gridpoint* up or down from where the derivative is requested.
            * Set derivstring to "upD"+"dirn" or "dnD"+"dirn", where "dirn" is an integer denoting direction. Example in 3D Cartesian coordinates (x,y,z):
                * the upwinded derivative operator $\partial_x$ corresponds to derivstring = "dupD0"
        * **Kreiss-Oliger dissipation derivatives**, where the center of the finite difference stencil corresponds to the point where the dissipation will be applied.
            * Set derivstring to "dKOD"+"dirn", where "dirn" is an integer denoting direction. Example in 3D Cartesian coordinates (x,y,z):
                * the Kreiss-Oliger derivative operator $\partial_z^\text{KO}$ corresponds to derivstring = "dKOD2"
    * **FDORDER**: an *optional* parameter that, if set to a positive even integer, overrides FD_CENTDERIVS_ORDER

Within NRPy+, `compute_fdcoeffs_fdstencl()` is only called from `FD_outputC()`. Regardless, this function provides a nice interface for evaluating finite difference coefficients, as shown below:

In [1]:
# Import the finite difference module
import finite_difference as fin  # NRPy+: Finite difference C code generation module

fdcoeffs, fdstencl = fin.compute_fdcoeffs_fdstencl("dDD00")
print(fdcoeffs)
print(fdstencl)

[-1/12, 4/3, -5/2, 4/3, -1/12]
[[-2, 0, 0, 0], [-1, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0]]


Interpreting the output, notice first that $\texttt{fdstencl}$ is a list of coordinate indices, where up to 4 dimension indices are supported (higher dimensions are possible and can be straightforwardly added, though be warned about [The Curse of Dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality)).

Thus NRPy+ found that for some function $u$, the fourth-order accurate finite difference operator at point $x_{i0}$ is given by

$$[\partial_{x}^{2} u]^\text{FD4}_{i0} = \frac{1}{\Delta x^{2}} \left[ -\frac{1}{12} \left(u_{i0-2,i1,i2,i3} + u_{i0+2,i1,i2,i3}\right) - \frac{5}{2}u_{i0,i1,i2,i3} + \frac{4}{3}\left(u_{i0-1,i1,i2,i3} + u_{i0+1,i1,i2,i3}\right)\right]$$

Notice also that multiplying by the appropriate power of $\frac{1}{\Delta x}$ term is up to the user of this function.

In addition, if the gridfunction $u$ exists on a grid that is less than four (spatial) dimensions, it is up to the user to truncate the additional index information.

<a id='exercise'></a>

### Step 1.a.i: Exercise: Using `compute_fdcoeffs_fdstencl()` \[Back to [top](#toc)\]
$$\label{exercise}$$

Using `compute_fdcoeffs_fdstencl()` write the necessary loops to output the finite difference coefficient tables in the Wikipedia article on [finite difference coefficients](https://en.wikipedia.org/wiki/Finite_difference_coefficients), for first and second centered derivatives (i.e., up to $\partial_i^2$)  up to eighth-order accuracy. [Solution, courtesy Brandon Clark](Tutorial-Finite_Difference_Derivatives-FDtable_soln.ipynb).

<a id='fdoutputc'></a>

## Step 1.b: The  `FD_outputC()` function \[Back to [top](#toc)\]
$$\label{fdoutputc}$$

**FD_outputC(filename,sympyexpr_list)**: C code generator for finite-difference expressions.

C codes that evaluate expressions with finite difference derivatives on numerical grids generally consist of three  components, all existing within a loop over "interior" gridpoints; at a given gridpoint, the code must
1. Read gridfunctions from memory at all points needed to evaluate the finite difference derivatives or the gridfunctions themselves.
2. Perform arithmetic, including computation of finite difference stencils.
3. Write the output from the arithmetic to other gridfunctions.

To minimize cache misses and maximize potential compiler optimizations, it is generally recommended to segregate the above three steps. FD_outputC() first analyzes the input expressions, searching for derivatives of gridfunctions. The search is very easy, as NRPy+ requires a very specific syntax for derivatives: 
* gf_dD0 denotes the first derivative of gridfunction "gf" in direction zero.
* gf_dupD0 denotes the upwinded first derivative of gridfunction "gf" in direction zero.
* gf_ddnD0 denotes the downwinded first derivative of gridfunction "gf" in direction zero.
* gf_dKOD2 denotes the Kreiss-Oliger dissipation operator of gridfunction "gf" in direction two.
Each time `FD_outputC()` finds a derivative (including references to the gridfunction directly \["zeroth"-order derivatives\]) in this way, it calls `compute_fdcoeffs_fdstencl()` to record the specific locations in memory from which the underlying gridfunction must be read to evaluate the appropriate finite difference derivative.

`FD_outputC()` then orders this list of points for all gridfunctions and points in memory, optimizing memory reads based on how the gridfunctions are stored in memory (set via parameter MemAllocStyle in the NRPy+ grid module). It then completes step 1. 

For step 2, `FD_outputC()` exports all of the finite difference expressions, as well as the original expressions input into the function, to outputC() to generate the optimized C code. Step 3 follows trivally from just being careful with the bookkeeping in the above steps.

`FD_outputC()` takes two arguments:
* **filename**: Set to "stdout" to print to screen. Otherwise specify a filename.
* **sympyexpr_list**: A single named tuple or list of named tuples of type "lhrh", where the lhrh type refers to the simple structure:
    * **lhrh(left-hand side of equation, right-hand side of the equation)**

Time for an example: let's compute 
$$
\texttt{output} = \text{phi_dDD00} = \partial_x^2 \phi(x,t),
$$
where $\phi$ is a function of space and time, though we only store its spatial values at a given time (*a la* the [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html), described & implemented in next the [Scalar Wave Equation module](Tutorial-Start_to_Finish-ScalarWave.ipynb)). 

As detailed above, the suffix $\text{_dDD00}$ tells NRPy+ to construct the second finite difference derivative of gridfunction $\texttt{phi}$ with respect to coordinate $xx0$ (in this case $xx0$ is simply the Cartesian coordinate $x$). Here is the NRPy+ implementation:

In [2]:
import sympy as sp               # SymPy, Python's core symbolic algebra package on which NRPy+ depends
from outputC import lhrh         # NRPy+: Core C code output module
import NRPy_param_funcs as par   # NRPy+: parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import finite_difference as fin  # NRPy+: Finite difference C code generation module

# Set the spatial dimension to 1
par.set_paramsvals_value("grid::DIM = 1")

# Register the input gridfunction "phi" and the gridfunction to which data are output, "output":
phi, output = gri.register_gridfunctions("AUX",["phi","output"])

# Declare phi_dDD as a rank-2 indexed expression: phi_dDD[i][j] = \partial_i \partial_j phi
phi_dDD = ixp.declarerank2("phi_dDD","nosym")

# Set output to \partial_0^2 phi
output = phi_dDD[0][0]

# Output to the screen the core C code for evaluating the finite difference derivative
fin.FD_outputC("stdout",lhrh(lhs=gri.gfaccess("out_gf","output"),rhs=output))

{
   /*
    * NRPy+ Finite Difference Code Generation, Step 1 of 2: Read from main memory and compute finite difference stencils:
    */
   /*
    *  Original SymPy expression:
    *  "const double phi_dDD00 = invdx0**2*(-5*phi/2 + 4*phi_i0m1/3 - phi_i0m2/12 + 4*phi_i0p1/3 - phi_i0p2/12)"
    */
   const double phi_i0m2 = aux_gfs[IDX2(PHIGF, i0-2)];
   const double phi_i0m1 = aux_gfs[IDX2(PHIGF, i0-1)];
   const double phi = aux_gfs[IDX2(PHIGF, i0)];
   const double phi_i0p1 = aux_gfs[IDX2(PHIGF, i0+1)];
   const double phi_i0p2 = aux_gfs[IDX2(PHIGF, i0+2)];
   const double FDPart1_Rational_5_2 = 5.0/2.0;
   const double FDPart1_Rational_1_12 = 1.0/12.0;
   const double FDPart1_Rational_4_3 = 4.0/3.0;
   const double phi_dDD00 = ((invdx0)*(invdx0))*(FDPart1_Rational_1_12*(-phi_i0m2 - phi_i0p2) + FDPart1_Rational_4_3*(phi_i0m1 + phi_i0p1) - FDPart1_Rational_5_2*phi);
   /*
    * NRPy+ Finite Difference Code Generation, Step 2 of 2: Evaluate SymPy expressions and write to main memory:
  

Some important points about the above code:
* The gridfunction PHIGF samples some function $\phi(x)$ at discrete uniform points in $x$, labeled $x_i$ at all points $i\in [0,N]$, so that 
$$\phi(x_i) = \phi_{i}=\text{in_gfs[IDX2(PHIGF, i)]}.$$ 
* For a *uniformly* sampled function with constant grid spacing (sample rate) $\Delta x$, $x_i$ is defined as $x_i = x_0 + i \Delta x$.
* The variable $\texttt{invdx0}$ must be defined by the user in terms of the uniform gridspacing $\Delta x$ as $\texttt{invdx0} = \frac{1}{\Delta x}$. 
     * *Aside*: Why do we choose to multiply by $1/\Delta x$ instead of dividing the expression by $\Delta x$, which would seem much more straightforward? 
         * *Answer*: as discussed in the [first part of the tutorial](Tutorial-Coutput__Parameter_Interface.ipynb), division of floating-point numbers on modern CPUs is far more expensive than multiplication, usually by a factor of ~3 or more.

<a id='latex_pdf_output'></a>

# Step 2: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Finite_Difference_Derivatives.pdf](Tutorial-Finite_Difference_Derivatives.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [3]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Finite_Difference_Derivatives")

Created Tutorial-Finite_Difference_Derivatives.tex, and compiled LaTeX file
    to PDF file Tutorial-Finite_Difference_Derivatives.pdf
